In [1]:
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import re
from selenium.common.exceptions import TimeoutException
from fake_useragent import UserAgent
from requests.exceptions import MissingSchema, ConnectTimeout
import datetime

In [2]:
URL = 'https://hh.ru'
profession = 'Программист'
PATH = 'data/'

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(URL)
driver.implicitly_wait(1)
driver.set_page_load_timeout(10)

headers = {
    'Accept': '*/*',
    'User-Agent': UserAgent().chrome
}



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\dm1tr\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [3]:
src = driver.page_source
soup = BeautifulSoup(src, 'lxml')

In [4]:
main_block = soup.find_all('div', class_='dashboard-tiles-item__body')

for b in main_block:
    print(b)
    print()

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Вакансии дня</div><div class="dashboard-tiles-item__salary">35 000 – 90 000 руб.</div></div><div class="dashboard-tiles-item__counter"><span>14 вакансий</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Компании дня</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"><span>1958<!-- --> <!-- -->вакансий</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Работа из дома</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"><span>37016<!-- --> <!-- -->вакансий</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles

In [5]:
for sub in main_block:
    if sub.find('div', class_='dashboard-tiles-item__title').text == profession:
        profession_selected_block = sub
print(profession_selected_block)

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Программист</div><div class="dashboard-tiles-item__salary">до <!-- -->495 000<!-- --> <!-- -->руб.</div></div><div class="dashboard-tiles-item__counter"><span>18206<!-- --> <!-- -->вакансий</span></div></div>


In [6]:
from selenium.webdriver.support.wait import WebDriverWait

professions = driver.find_elements(by=By.CLASS_NAME, value='dashboard-tiles-item__title')

links_to_profession = list()

if professions:
    professions[0].click()
    professions[0].click()

for elem in professions:
    if elem.text == profession:
        elem.click()
        element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, "//*[@class='multiple-column-list multiple-column-list_wide']"))
        )
        soup = BeautifulSoup(driver.page_source, 'lxml')
        ul = soup.find(class_='multiple-column-list multiple-column-list_wide')
        for li in ul:
            links_to_profession.append(li.a.get('href'))

In [7]:
def scrap_offer(link, data):
    print(link)
    try:
        req = requests.get(link, headers=headers)
        soup = BeautifulSoup(req.text, parser='lxml')
        title = soup.find('h1', class_='bloko-header-1').text

        reviews_button = soup.find('button', class_='_2kMmwZ7___bloko-link _2iCEfb3___bloko-link_pseudo')

        approved = True if soup.find('span', class_='bloko-icon bloko-icon_done bloko-icon_initial-action') else False
        in_top = True if soup.find('span',
                                   class_='bloko-icon bloko-icon_employer-hh-rating bloko-icon_initial-unique') else False

        experience = soup.find('span', {'data-qa': 'vacancy-experience'})
        experience = experience.text if experience else None

        busyness = soup.find('p', {'data-qa': 'vacancy-view-employment-mode'})
        busyness = busyness.text if busyness else None

        if reviews_button:
            reviews = reviews_button.span.text.split()[0]
            rating = soup.find('div',
                               class_='_2uXXPFp___bloko-text _3Iu9HPD___bloko-text_extra-large mmwQivW___bloko-text_strong').text
        else:
            reviews = None
            rating = None

        data['busyness'].append(busyness)
        data['experience'].append(experience)
        data['in_top'].append(in_top)
        data['reviews_count'].append(reviews)
        data['avg_rating'].append(rating)
        data['title'].append(title)
        data['approved'].append(approved)

    except MissingSchema:
        print('Shema error')
        data['salary'].pop(-1)
        data['prof_type'].pop(-1)
    except AttributeError:
        print('None type')
        data['salary'].pop(-1)
        data['prof_type'].pop(-1)
    except ConnectTimeout:
        print('Connect timeout')
        data['salary'].pop(-1)
        data['prof_type'].pop(-1)

In [9]:
data = {'title': list(), 'prof_type': list(), 'salary': list(), 'reviews_count': list(),
        'avg_rating': list(), 'approved': list(), 'in_top': list(), 'experience': list(),
        'busyness': list()}
page = 0

for link in links_to_profession[::]:
    req = requests.get(f'{URL}{link}&page={page}', headers=headers)
    print(req.status_code)
    while req.status_code == 200:
        soup = BeautifulSoup(req.text, parser='lxml')

        prof_type = soup.find('h1', class_='bloko-header-section-3').text
        prof_type = re.compile('«.*»').findall(prof_type)[0][1:-1]

        for offer in soup.find_all('div', class_='vacancy-serp-item'):
            raw_salary = offer.find('span', class_='bloko-header-section-3')

            data['prof_type'].append(prof_type)

            if not raw_salary:
                data['salary'].append(None)
            else:
                salary = re.compile('\d+').findall(raw_salary.text.replace(' ', ''))

                if len(salary) > 1:
                    data['salary'].append(f'{salary[0]}-{salary[1]}')
                elif 'от' in raw_salary.text:
                    data['salary'].append(f'от {salary[0]}')
                elif 'до' in raw_salary.text:
                    data['salary'].append(f'до {salary[0]}')
                else:
                    data['salary'].append(None)

            offer_link = offer.find('a', class_='bloko-link').get('href')
            try:
                scrap_offer(offer_link, data)
            except TimeoutException:
                print('Timeout')
                data['salary'].pop(-1)
                data['prof_type'].pop(-1)

        page += 1
        req = requests.get(f'{URL}{link}&page={page}', headers=headers)
    page = 0

200
https://hh.ru/vacancy/54814745?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1
https://hh.ru/vacancy/54839889?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1
https://hh.ru/vacancy/54712154?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1
https://hh.ru/vacancy/54841195?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1
/vacancies/programmist
Shema error
https://hh.ru/vacancy/54721618?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82%201%D0%A1
https://hh.ru/vacancy/54635259?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80

In [ ]:
data['salary'].pop(-1)
data['prof_type'].pop(-1)

In [10]:
for k, v in data.items():
    print(len(v))

12343
12343
12343
12343
12343
12343
12343
12343
12343


In [11]:
df = pd.DataFrame(data=data)
df

,title,prof_type,salary,reviews_count,avg_rating,approved,in_top,experience,busyness
0,Ведущий программист 1С,Программист 1С,200000-250000,None,None,True,False,1–3 года,"Полная занятость, полный день"
1,"Программист 1С (гибридный график, поддержка)",Программист 1С,90000-120000,4,"3,5",True,False,3–6 лет,"Полная занятость, полный день"
2,Программист 1С,Программист 1С,от 200000,None,None,True,False,3–6 лет,"Полная занятость, полный день"
3,Программист 1С,Программист 1С,до 300000,6,"4,7",True,False,3–6 лет,"Полная занятость, удаленная работа"
4,1С ведущий консультант,Программист 1С,от 250000,None,None,True,False,3–6 лет,"Полная занятость, удаленная работа"
...,...,...,...,...,...,...,...,...,...
12338,Middle Frontend Разработчик (React),Junior-программист,180000-250000,3,"4,6",True,False,3–6 лет,"Полная занятость, гибкий график"
12339,Senior/Middle iOS Developer,Junior-программист,200000-400000,None,None,False,False,3–6 лет,"Полная занятость, гибкий график"
12340,Ruby on Rails Developer / Программист / Разраб...,Junior-программист,до 80000,None,None,True,False,1–3 года,"Полная занятость, полный день"
12341,Программист 1С,Junior-программист,None,25,"4,1",True,False,1–3 года,"Полная занятость, полный день"


In [57]:
df['avg_rating'].unique()

array([None, '3,5', '4,6', '4,5', '5,0', '3,7', '4,4', '4,9', '3,9',
       '4,7', '3,6', '4,8', '4,1', '4,2', '3,4', '4,3', '3,8', '4,0',
       '3,0', '3,3', '3,1', '2,9'], dtype=object)

In [12]:
df.to_csv(PATH + datetime.datetime.now().strftime("%Y-%m-%d=%H-%M-%S") + '.csv', index=False)


In [14]:
df1 = pd.read_csv(PATH + 'last.csv')

In [15]:
df1

,title,prof_type,salary,reviews_count,avg_rating,approved,in_top,experience,busyness
0,Ведущий программист 1С,Программист 1С,200000-250000,NaN,NaN,True,False,1–3 года,"Полная занятость, полный день"
1,"Программист 1С (гибридный график, поддержка)",Программист 1С,90000-120000,4.0,"3,5",True,False,3–6 лет,"Полная занятость, полный день"
2,Программист 1С,Программист 1С,от 200000,NaN,NaN,True,False,3–6 лет,"Полная занятость, полный день"
3,Программист 1С,Программист 1С,до 300000,6.0,"4,7",True,False,3–6 лет,"Полная занятость, удаленная работа"
4,1С ведущий консультант,Программист 1С,от 250000,NaN,NaN,True,False,3–6 лет,"Полная занятость, удаленная работа"
...,...,...,...,...,...,...,...,...,...
12338,Middle Frontend Разработчик (React),Junior-программист,180000-250000,3.0,"4,6",True,False,3–6 лет,"Полная занятость, гибкий график"
12339,Senior/Middle iOS Developer,Junior-программист,200000-400000,NaN,NaN,False,False,3–6 лет,"Полная занятость, гибкий график"
12340,Ruby on Rails Developer / Программист / Разраб...,Junior-программист,до 80000,NaN,NaN,True,False,1–3 года,"Полная занятость, полный день"
12341,Программист 1С,Junior-программист,NaN,25.0,"4,1",True,False,1–3 года,"Полная занятость, полный день"
